# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [162]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

# получаем общую информацию о таблице
print(df.info())

# корректируем названия столбцов
df.set_axis(['children','days_employed','age','education','education_id','family_status','family_status_id','gender','income_type','delay','monthly_income','purpose'],axis = 'columns',inplace = True)

# убираем лишние столбцы с идентификаторами
df = df.drop(['family_status_id','education_id'], 1)

print(df.head(10))
print()
print(df['education'].value_counts()) # значения в столбце education имеют прописные буквы
print()
print(df['children'].value_counts()) # children имеет отрицательные значения (-1) и заведомо завышенные (20)
print()
print('Минимальный возраст:', df['age'].min()) # нулевые значения в столбце age
print('Максимальный возраст:', df['age'].max())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
   children  days_employed  age education     family_status gender  \
0         1   -8437.673028   42    высшее   женат / замужем      F   
1         1   -4024.803754   36   среднее   женат / замужем      F   
2         0   -5623.422610   33   Среднее   женат / замужем      M   
3         3   -4124.747207   32   среднее  

**Вывод**

Мы проанализировали таблицу методами info() и value_counts(), переменивали столбцы методом set_axis() и удалили лишние столбцы методом drop().
Таблица включает 21525 записей. Каждая строчка - один заемщик, охарактеризованный:
 - 4-мя количественными переменными (кол-во детей, трудовой стаж, возраст и среднемесячный доход)
 - 5-ю категориальными переменными (образование, семейное положение, пол, тип занятости и цель получения кредита)
 - 1-й логической переменной (факт просроченного платежа)
 
В ходе анализа мы обнаружили следующие "проблемные места" в предоставленных данных:
 - Отрицательные и завышенные значения в столбце children
 - Не унифицированные по написанию значения в столбце education
 - Отрицательные и дробные значения в столбце days_employed
 - Завышенные значения days_employed (насчитывающие сотни лет трудового стажа) для заемщиков-пенсионеров и безработных
 - Нулевые значения в столбце age
 - Пропуски в столбцах days_employed и monthly_income

## Шаг 2. Предобработка данных

### Обработка пропусков

In [163]:
df['education'] = df['education'].str.lower() #убираем прописные буквы из столбца education
print(df['education'].value_counts()) #теперь данные пригодны для работы

# обрабатываем минусовые и завышенные значения days_employed, исходя из гипотезы, что отрицательные данные корректны по модулю, а в завышенных данных представлены дни, переведенные в часы
def edit_days_employed(row):
    if row['income_type'] != 'пенсионер' and row['income_type'] != 'безработный':
        if row['days_employed'] <0:
            return(row['days_employed']*(-1))
    else:
        return(row['days_employed']/(24))

df['days_employed'] = df.apply(edit_days_employed, axis=1)

print(df.isna().sum()) # обнаружено равное количество пропусков в столбцах monthly_income и days_employed

# пишем функцию, которая будет заменять пустые и нулевые значения на медианные по определенной группе

def fillna_with_median(df,na_value,group_values):
    df_median = (df.groupby(group_values)).agg({na_value: 'median'}).rename(columns = {na_value:'median_value'})    
    df = df.merge(df_median, on = group_values)
    df.loc[(df[na_value].isna()) | (df[na_value] == 0),na_value] = df.loc[(df[na_value].isna()) | (df[na_value] == 0),'median_value']
    df = df.drop('median_value', 1)
    df[na_value] = df[na_value].fillna(df[na_value].mean()) #заполняем оставшиеся нулевые значения медианой по общей выборке, если они присутствуют в базе в единственном числе
    return (df)

# для корректного расчета медианы по зарплате и стажу нам необходимо заполнить пропущенные значения возраста, а также ввести ранжирование заемщиков по возрасту

df = fillna_with_median(df,'age',['education','income_type'])

# пишем функцию для ранжирования заемщиков по возрасту
def define_age_group(row):
    age_groups = [
        ['от 18 до 23 лет',18,23],
        ['от 24 до 29 лет',24,29],
        ['от 30 до 39 лет',30,39],
        ['от 40 до 54 лет',40,54],
        ['от 55 лет',55,df['age'].max()]
    ]
    
    for i in age_groups:
        if i[1] <= row['age'] <= i[2]:
            return(i[0])
        
df['age_group'] = df.apply(define_age_group,axis=1)

print(df[df['days_employed'].isna()]['days_employed'].count())
       
# заполняем пропуски функцией fillna_with_median
df = fillna_with_median(df,'monthly_income',['education','income_type','age_group'])
df = fillna_with_median(df,'days_employed',['education','income_type','age_group'])

print(df['children'].value_counts()) #обнаруживаем мусорные значения в графе children - (-1) и 20. Нужна замена
df.loc[df['children'] == 20, 'children'] = 2 #предполагаем, что к данным значениям добавлен ноль из-за технической ошибки или человеческого фактора
df.loc[df['children'] == -1, 'children'] = 1 #предполагаем, что данные значения стали отрицательными из-за технической ошибки или человеческого фактора
print(df['children'].value_counts()) #проверка замены




среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
children             0
days_employed     2174
age                  0
education            0
family_status        0
gender               0
income_type          0
delay                0
monthly_income    2174
purpose              0
dtype: int64
0
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


**Вывод**

В данном блоке мы скорректировали следующие данные:
 - Перевели весь столбец <b>'education'</b> к единому формату написания только строчными буквами, тем самым избавившись от множества дубликатов
 - Скорректировали значения в столбце <b>'days_employed'</b>, исходя из гипотезы о том, что отрицательные значения корректны по модулю, а завшенные значения представлены в часах
 - Заполнили пропуски в столбцах <b>'days_employed'</b>, <b>'age'</b> и <b>'monthly_income'</b>, заменив их на соответствующие медианные значения от группировки по типу дохода/образованию/возрастной группе
 - Скорректировали отрицательные и завышенные значения в столбце <b>'children'</b>

### Замена типа данных

In [164]:
df['days_employed'] = df['days_employed'].astype(int) # присваиваем всем переменным float тип int
df['monthly_income'] = df['monthly_income'].astype(int) 
df['age'] = df['age'].astype(int)

**Вывод**

С применением метода <b>astype()</b> мы перевели все значения типа <b>float</b> в <b>int</b>

### Обработка дубликатов

In [165]:
print(df.duplicated().sum()) #обнаруживаем 71 дублированную строчку. С учетом объема данных маловероятно, что это не дубликаты
print(df[df.duplicated()].head(20))
df = df.drop_duplicates().reset_index(drop = True) # удаляем с обновлением индексов
print(df.duplicated().sum()) #дубликаты отсутствуют

71
      children  days_employed  age education          family_status gender  \
951          1           1496   34    высшее       гражданский брак      F   
1445         1           1496   32    высшее        женат / замужем      F   
1603         1           1496   30    высшее        женат / замужем      F   
2065         0           1081   29    высшее        женат / замужем      M   
3054         0           1572   30   среднее        женат / замужем      M   
3414         2           1572   34   среднее        женат / замужем      F   
3781         1           1572   37   среднее        женат / замужем      F   
3951         0           1572   35   среднее       гражданский брак      F   
4330         2           1572   39   среднее       гражданский брак      F   
5381         0           1951   41   среднее        женат / замужем      F   
6859         0           1951   47   среднее        женат / замужем      F   
7054         1           1951   44   среднее        женат / з

**Вывод**

Мы обнаружили <b>71</b> дубликат методом <b>duplicated()</b> и приняли решение удалить их методом <b>drop_duplicates()</b> с обновлением индексов.
В вывденных на экран дубликатах нет явной корреляции данных, поэтому сложно объяснить их происхождение. Это может быть как техническая ошибка, так и человеческий фактор.

### Лемматизация

In [166]:
from pymystem3 import Mystem
m = Mystem()

PURPOSE_DICT = {
        'автомобиль': 'автомобиль',
        'образование': 'образование',
        'недвижимость': ['недвижимость','жилье','жильё'],
        'свадьба': 'свадьба'
    }

  
def add_purpose_groups(row):
    lemmas = m.lemmatize(row['purpose'])
    for word in lemmas:
        for purpose_type in PURPOSE_DICT:
            if word in PURPOSE_DICT[purpose_type]:
                return purpose_type
    return('неизвестная цель')

df['purpose_group'] = df.apply(add_purpose_groups,axis=1)

print('Неизвестных категорий целей получения кредита: ', df[df['purpose_group'] == 'неизвестная цель']['purpose_group'].count()) # проверяем количество нулевых значений

Неизвестных категорий целей получения кредита:  0


**Вывод**

Из присутствующих в базе категорий я смог выделить 4 типа целей для кредитования: <b>покупка автомобиля, оплата образования, недвижимость</b> и <b>организация свадьбы</b>. Поскольку в данных нет единого формата указания цели кредита, мы проводим лемматизацию каждой строчки и проверяем наличие каждого слова в словаре <b>PURPOSE_DICT</b>. После нахождения слова в словаре функция возвращает соответствующий ключ.

### Категоризация данных

In [167]:
# создадим цикл для оценки количества заемщиков с разным доходом с шагом в 50 тыс руб, чтобы определиться с группировкой заемщиков по уровню дохода

for i in range(df['monthly_income'].min(),df['monthly_income'].max(),50000):
    count = df[(df['monthly_income'] >= i) & (df['monthly_income'] <= (i+50000))]['monthly_income'].count()
    print('Кол-во заемщиков с доходом от', i, ' до ', i+50000, ' - ', count)

# создаем функции для ранжирования заемщиков по уровню дохода и наличию детей

def add_income_group(row):
    if row['monthly_income'] < 50000:
        return (pd.Series(['до 50 тыс руб', 0]))
    if 50000 <= row['monthly_income'] < 120000:
        return(pd.Series(['от 50 до 120 тыс руб', 1]))
    if 120000 <= row['monthly_income'] < 250000:
        return(pd.Series(['от 120 до 250 тыс руб', 2]))
    if 250000 <= row['monthly_income'] < 450000:
        return(pd.Series(['от 250 до 450 тыс руб', 3]))
    if 450000 <= row['monthly_income']:
        return(pd.Series(['более 450 тыс руб', 4]))

def define_children(row):
    if row['children'] > 0:
        return('Есть дети')
    else:
        return('Нет детей')

df[['income_group','income_group_index']] = df.apply(add_income_group,axis=1)
df['has_children'] = df.apply(define_children,axis=1)


Кол-во заемщиков с доходом от 20667  до  70667  -  1526
Кол-во заемщиков с доходом от 70667  до  120667  -  5756
Кол-во заемщиков с доходом от 120667  до  170667  -  6656
Кол-во заемщиков с доходом от 170667  до  220667  -  3534
Кол-во заемщиков с доходом от 220667  до  270667  -  1856
Кол-во заемщиков с доходом от 270667  до  320667  -  916
Кол-во заемщиков с доходом от 320667  до  370667  -  492
Кол-во заемщиков с доходом от 370667  до  420667  -  285
Кол-во заемщиков с доходом от 420667  до  470667  -  147
Кол-во заемщиков с доходом от 470667  до  520667  -  96
Кол-во заемщиков с доходом от 520667  до  570667  -  58
Кол-во заемщиков с доходом от 570667  до  620667  -  30
Кол-во заемщиков с доходом от 620667  до  670667  -  25
Кол-во заемщиков с доходом от 670667  до  720667  -  24
Кол-во заемщиков с доходом от 720667  до  770667  -  8
Кол-во заемщиков с доходом от 770667  до  820667  -  5
Кол-во заемщиков с доходом от 820667  до  870667  -  9
Кол-во заемщиков с доходом от 870667  до

**Вывод**

В данном блоке, а также в предыдущих частях проекта мы ввели следующие категории заемщиков:
 - <b>По наличию / отсутствию детей</b> - анализируя данные в столбце children
 - <b>По уровню дохода</b> - анализируя данные в столбце monthly_income. Также добавлен индекс для удобства сортировки
 - <b>По целям получения кредита</b> - путем лемматизации столбца purpose

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [168]:
print(df.groupby('has_children')['delay'].mean())
print()
print(df.groupby('children')['delay'].mean())
print()
print(df['children'].value_counts())

has_children
Есть дети    0.092082
Нет детей    0.075438
Name: delay, dtype: float64

children
0    0.075438
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
Name: delay, dtype: float64

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


**Вывод**

Наличие детей однозначно негативно влияет на вероятность погашения кредита в срок. Шансы наличия задолженности у категорий "есть дети" и "нет детей" <b>9.2%</b> против <b>7.5%</b> соответственно.

Стоит отметить, что семьи с 1-2 детьми отличаются наихудшей финансовой дисциплиной (<b>9.2</b> и <b>9.5%</b> должников соответственно), зато многодетные заемщики с 3-мя детьми практически приближаются к надежности к бездетным (<b>8.2%</b>). Возможно, это связано с финансовой помощью со стороны государства.

Данные по заемщикам с 4-мя (<b>9.8%</b> должников) и 5-ю (<b>0%</b> должнников) скорее всего искажены из-за небольшого объема выборки и по ним трудно делать какие-то выводы.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [169]:
df_groupby_family_status = df.groupby(by = 'family_status')
print(df_groupby_family_status['delay'].mean().sort_values(ascending = False))

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093471
женат / замужем          0.075452
в разводе                0.071130
вдовец / вдова           0.065693
Name: delay, dtype: float64


**Вывод**

Заемщики, никогда не состоявщие в официальном браке, отличаются наихудшей финансовой дисциплиной (вероятность наличия задолженности <b>9.75%</b>). При этом нахождение заемщика в гражданском браке практически не оказывает влияния на вероятность задолженности (<b>9.35%</b>).

Заемщики, которые состоят или когда-либо состояли в официальном браке - гораздо надежнее. Шанс задолженности для такой категории заемщиков находится в диапазоне от <b>6.5%</b> до <b>7.5%</b>, причем наиболее ответственные - вдовы/вдовцы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [170]:
print(df.groupby(['income_group_index','income_group'])['delay'].mean())
print()
print(df['income_group'].value_counts())


income_group_index  income_group         
0                   до 50 тыс руб            0.061828
1                   от 50 до 120 тыс руб     0.081982
2                   от 120 до 250 тыс руб    0.084274
3                   от 250 до 450 тыс руб    0.070968
4                   более 450 тыс руб        0.054217
Name: delay, dtype: float64

от 120 до 250 тыс руб    11427
от 50 до 120 тыс руб      6843
от 250 до 450 тыс руб     2480
до 50 тыс руб              372
более 450 тыс руб          332
Name: income_group, dtype: int64


**Вывод**

Наиболее дисциплинированы заемщики, чей уровень дохода проходит по верхней и (как ни парадоксально) по нижней границе выборки. Клиенты с доходами <b>менее 50 тыс руб</b> и <b>более 450 тыс руб</b> попадают в должники в <b>6.2%</b> и <b>5.4%</b> случаев соответственно.

Клиенты со средним уровнем дохода (<b>от 50 до 250 тыс руб</b>) представляют наибольший риск для банка: трудности с выплатой кредита <b>8.2-8.4%</b> случаев. 

Клиенты с доходом выше среднего (<b>от 250 до 450 тыс руб</b>) демонстрируют среднюю платежную дисциплину: не возвращают кредит в срок <b>7.1%</b> из выборки.

- Как разные цели кредита влияют на его возврат в срок?

In [171]:
df_groupby_purpose = df.groupby(by = 'purpose_group')
print(df_groupby_purpose['delay'].mean().sort_values(ascending = False))

purpose_group
образование     0.092200
автомобиль      0.089747
свадьба         0.084319
недвижимость    0.070976
Name: delay, dtype: float64


**Вывод**

Наиболее проблемны кредиты, взятые с целью получения <b>образования</b>: <b>9.2%</b> заемщиков из выборки имеют проблемы с погашением кредита в срок. Хотя вероятность задолженности по кредитам на <b>покупку автомобиля</b> или <b>проведения свадьбы</b> достаточно близка (<b>8.96%</b> и <b>8.38%</b> соответственно).

Наиболее надежны заемщики, берущие кредит на <b>приобретение недвижимости</b>. Они имеют заложенность всего <b>7.1%</b>. Скорее всего для такой категории заемщиков менее свойственны спонтанные, необдуманные решения.

Резюмируя, только кредиты на приобретение недвижимости отличаются высокой (на фоне общей выборки) надежностью. Во всех остальных случаях цель кредитования практически не влияет на шанс погашения кредита вовремя.

## Шаг 4. Общий вывод

Мы изучили представленную базу данных, избавились от пропусков, скорректировали "мусорные" значения, провели категоризацию заемщиков по нескольким категориям и провели анализ влияния различных факторов на вероятность выплаты кредита. Полученные значения свидетельствуют о том, что все 4 фактора - семейное положение, цель кредита, наличие детей и доход заемщика - существенным образом (до 2 процентных пунктов) влияют на вероятность попадания заемщика в должники.

Резюмируя сделанные ранее в проекте выводы, можно отметить следующие факторы, которые позитивно влияют на вероятность выплаты кредита в срок:
   - Заемщик состоит, либо ранее состоял в браке
   - У заемщика нет детей
   - Цель получения кредита - приобретение недвижимости
   - Доход заемщика либо менее 50 тыс. руб в месяц, либо превышает 450 тыс. руб в месяц
 
Факторы, негативно влияющие на вероятность выплаты кредита в срок:
   - Заемщик не состоит и никогда не состоял в официальном браке
   - Заемщик является родителем 1-2 детей
   - Доход заемщика находится в диапазоне от 50 до 250 тыс руб

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

In [172]:
print('Hello, World!')

Hello, World!
